In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoTokenizer

/home/cosmoquester/aibookathon/env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
summarize_model_name: str = "digit82/kobart-summarization"
summarize_num_beams: int = 4
sum_input_max_length: int = 512
sum_max_token_length: int = 128

model_name: str = "../model-00epoch-4000steps-4.3906ppl"
tokenizer_name: str = model_name
prompt_max_length = 128
max_length: int = 512
device: str = "cuda"
auth_token: str = None

num_generate_characters: int = 20500
use_n_recent_summarizations: int = 10

In [3]:
tokenizer_kwargs = {"padding_side": "left", "truncation_side": "left"}
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_auth_token=auth_token, **tokenizer_kwargs)
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=auth_token).to(device)

In [4]:
tokenizer_kwargs = {"padding_side": "left", "truncation_side": "left"}
summarize_tokenizer = AutoTokenizer.from_pretrained(summarize_model_name, use_auth_token=auth_token, **tokenizer_kwargs)
summarize_model = AutoModelForSeq2SeqLM.from_pretrained(summarize_model_name).to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [5]:
def datum_to_string(datum, use_n_summary: int = 10):
    text = f"제목: {datum['title']}\n"

    summarizations = datum.get("summarizations")
    if summarizations:
        summarizations = summarizations[-use_n_summary:]
        summarization = " ".join(summarizations).replace("\n", " ")
        text = f"요약: {summarization}\n" + text
    return text

def segment_by_anychar(text: str, delimiters: str):
    segments = []
    last_index = 0
    for i, char in enumerate(text):
        if char in delimiters:
            segments.append(text[last_index: i + 1])
            last_index = i + 1
    return segments

In [6]:
initial_input_example = {
    "title": "나의 빛과 어둠", 
    "summarizations": [], 
    "content": ""
}
seed: int = 42
backup_indices = [0, 11, 4, 25] # Used for replay already found indices or continue writing from special part

In [7]:
from IPython.display import clear_output
from copy import deepcopy

torch.manual_seed(seed)
input_example = deepcopy(initial_input_example)
select_indices = []

while len(input_example["content"]) < num_generate_characters:
  prompt = datum_to_string(input_example)
  input_ids = tokenizer(
          [prompt],
          add_special_tokens=False,
          max_length=prompt_max_length,
          padding="longest",
          truncation=True,
          return_tensors="pt",
          return_token_type_ids=False,
          return_attention_mask=False,
  )["input_ids"].to(device)

  output = model.generate(input_ids, 
      max_length, 
      do_sample=True,
      num_return_sequences=1,
      # repetition_penalty=1.2,
      # num_beams=5,
      # temperature=2.0,
      # top_p=0.99,
      # top_k=3,
      pad_token_id=tokenizer.pad_token_id,
      use_cache=True,
  )
  output = output.squeeze(dim=0)[input_ids.size(1):]

  text = tokenizer.decode(output, skip_special_tokens=True)
  segments = segment_by_anychar(text, ".?!\n'\"")
  segments.insert(0, "")

  print("[TITLE]")
  print(input_example["title"])
  print("\n[SUMMARIZATIONS]")
  for i, summarization in enumerate(input_example["summarizations"], start=ord("A")):
    print(f"[{chr(i)}] {summarization}")
  print("\n[KEEP CONTENT]")
  print(input_example["content"])
  print("\n[CUR SENTENCES]")
  for i, segment in enumerate(segments):
    print(f"[{i}] {segment}")
  print("[-1] ※ EXIT: 더 이상 문장을 생성하지 않습니다.")

  if backup_indices:
    select_index = backup_indices.pop(0)
  else:
    while True:
      try:
        select_index = int(input("Select last sentence index: "))
        break
      except:
        continue

  select_indices.append(select_index)
  if select_index < 0:
    break

  clear_output(wait=True)
  print(flush=True, end="")
  
  selected_segment = segments[:select_index + 1]
  selected_text = "".join(selected_segment)

  if not selected_text:
    continue

  input_example["content"] += selected_text
  sum_input_ids = summarize_tokenizer(
          [selected_text],
          add_special_tokens=False,
          max_length=sum_input_max_length,
          padding="longest",
          truncation=True,
          return_tensors="pt",
          return_token_type_ids=False,
          return_attention_mask=False,
  )["input_ids"].to(device)

  sum_outputs = summarize_model.generate(sum_input_ids, 
      sum_max_token_length, 
      num_beams=summarize_num_beams,
      num_return_sequences=1,
      eos_token_id=summarize_tokenizer.eos_token_id,
      pad_token_id=summarize_tokenizer.pad_token_id,
      use_cache=True,
  )

  summarization = summarize_tokenizer.decode(sum_outputs.squeeze(dim=0), skip_special_tokens=True)
  input_example["summarizations"].append(summarization)
  input_example["summarizations"] = input_example["summarizations"][-use_n_recent_summarizations:]

[TITLE]
나의 빛과 어둠

[SUMMARIZATIONS]
[A] 유난히 영화를 좋아했던 나는 내 인생의 어느 순간이 그런 빛의 상태였을까 궁금하다.
[B] 어떤 삶은 기억으로 남고, 어떤 삶은 기억으로 남고, 어떤 삶은 추억으로 남는다.
[C] 하율이는 내 첫사랑이자 전부고, 영원이고, 내 세상의 전부를 가져서, 더 이상 그 아이가 아닌 다른 아이가 태어나도 네게 사랑을 줄 거야 이라며 말을 잘하지 못해 말을 한 사람도 잘 모르고 들은 사람도 잘 기억하지 못한다.
[D] ‘세상을 다 잃었다고 생각한 사람이 갑자기 모든 것을 내어버리고선, 세상의 전부를 손에 쥐기라도 한 듯 행복해하는 것은 나의 빛과 나의 어둠을 알고 있기에 가능한 일이고 세상의 전부를 내 품에 안은 채 날마다 숨 쉬고 웃을 수 있는 것도 나의 빛과 나의 어둠을 알고있기에 가능하다.
[E] 내가 원하는 것을 모두 가지려면 내가 좋아하는 것들을 모두 버리고 희생해야 한다.
[F] 내가 어렸을 때 부터 내 옆에서 나로 인해 기뻐하는 모습을 보며 나의 재능이라고 소개 받았으며 더 많은 글과 그림을 통해 재능을 사람들에게 선보이고 싶다고 말했다.

[KEEP CONTENT]
내가 좋아하는 영화
'이터널 선샤인'
사랑은 무엇인가에 대해 생각하게 될 때 꼭 보는 영화지만 볼 때마다 뭔가 가슴이 두근거린다. 나는 사람이든 사물이든 늘 그것이 가진 고유한 빛 때문에 혼란스러운 상태에 놓인다. 나는 내 인생의 어느 순간이 그런 빛의 상태였을까 궁금했다. 영화의 시작 부분에서 이터널이라는 낱말이 눈에 들어오는 순간 나는 내 인생의 어느 한 부분이 떠올랐다.
어린 시절 나는 유난히 영화를 좋아했다. 그것은 내 생애 최초의 영화 관람 경험인 동시에 내가 영화에 푹 빠져들 수 있다는 것을 증명한 최초의 경험이었을 것이다.어떤 삶은 기억으로 남고, 어떤 삶은 추억으로 남는다. 그건 아마도 빛과 어둠의 농도 차이가 아닐까 싶다. 나의 삶에 빛이 더 많았다면 좋았겠지만, 그렇지 않을 수도 있다. 어쩌면 나의 기억이 빛

In [8]:
print("Initial Input Example:", initial_input_example)
print("Seed:", seed)
print("Selected Indices:", select_indices)
print("※ 위 정보를 이용하면 똑같은 결과를 다시 재현할 수 있습니다. (selected indices를 backup indices로 사용)")

Initial Input Example: {'title': '나의 빛과 어둠', 'summarizations': [], 'content': ''}
Seed: 42
Selected Indices: [0, 11, 4, 25, 14, 14, 6, -1]
※ 위 정보를 이용하면 똑같은 결과를 다시 재현할 수 있습니다. (selected indices를 backup indices로 사용)


In [9]:
print("※ 혹시나 중간과정을 스킵하고 이어서 추론을 원한다면 현재 `input_example`을 복원하고 seed를 아래의 값으로 설정하세요.")
print("Current Seed:", torch.seed())

※ 혹시나 중간과정을 스킵하고 이어서 추론을 원한다면 현재 `input_example`을 복원하고 seed를 아래의 값으로 설정하세요.
Current Seed: 4987955390658751426
